# Part I. ETL Pipeline for Pre-Processing the Files

Make sure to read the README.md of the project: 
[github.com/Aleaume/Udacity_DataEng_P2](https://github.com/Aleaume/Udacity_DataEng_P2)

#### Import Python packages 

In [49]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [50]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [51]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [52]:
# check the number of rows in your csv file

with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Creating the Apache Cassandra Keyspace and its Tables

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

See image in folder "images"

#### Creating a Cluster

In [53]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [54]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [55]:
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries.



### Query 1: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4




In [56]:
query = "CREATE TABLE IF NOT EXISTS sessions "
query = query + "(sessionId int, itemInSession int, artist text, duration float, song text, PRIMARY KEY (sessionId, itemInSession))"

session.execute(query)
                    

In [57]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO sessions (sessionid, iteminsession, artist, duration, song)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"

        session.execute(query, (int(line[8]), int(line[3]), line[0], float(line[5]), line[9]))
        

In [58]:
#commented line give the quality check result with checking parameters

#query_case_1 = "SELECT * FROM sessions "
query_case_1 = "SELECT artist, song, duration FROM sessions "
query_case_1 +="WHERE sessionid = 338 AND iteminsession = 4"

data = []

try:
    rows = session.execute(query_case_1)
except Exception as e:
    print(e)
    
for row in rows:
    #print (row.artist, row.song, row.duration, row.sessionid, row.iteminsession)

    data.append(
    
        {
            'Artist': row.artist,
            'Song': row.song,
            'Duration': row.duration
        }
    
    )

df = pd.DataFrame(data)
df = df[['Artist','Song','Duration']]

df

,Artist,Song,Duration
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [59]:
query = "CREATE TABLE IF NOT EXISTS users "
query += "(userid int, sessionid int, iteminsession int, artist text, song text, firstname text, lastname text, PRIMARY KEY ((userid, sessionid),iteminsession))"

session.execute(query)
                    

In [60]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users (userid, sessionid, iteminsession, artist, song, firstname, lastname)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"

        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [61]:
#commented line give the quality check result with checking parameters

#query_case_2 = "SELECT * FROM users "
query_case_2 = "SELECT artist, song, firstname, lastname FROM users "
query_case_2 +="WHERE userid = 10 AND sessionid = 182 "

data = []

try:
    rows = session.execute(query_case_2)
except Exception as e:
    print(e)
    
for row in rows:
    #print (row.artist, row.song, row.firstname, row.lastname, row.userid, row.sessionid, row.iteminsession)
    
    data.append(
    
        {
            'Artist': row.artist,
            'Song': row.song,
            'Firstname': row.lastname,
            'Lastname': row.firstname
        }
    
    )

df = pd.DataFrame(data)
df = df[['Artist','Song','Firstname','Lastname']]

df

,Artist,Song,Firstname,Lastname
0,Down To The Bone,Keep On Keepin' On,Cruz,Sylvie
1,Three Drives,Greece 2000,Cruz,Sylvie
2,Sebastien Tellier,Kilometer,Cruz,Sylvie
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Cruz,Sylvie


### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [62]:
query = "CREATE TABLE IF NOT EXISTS songs "
query = query + "(song text, userid int, sessionid int, firstname text, lastname text, PRIMARY KEY (song, userid))"

session.execute(query)
                    

In [63]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs (song, userid, sessionid, firstname, lastname)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
        session.execute(query, (line[9], int(line[10]), int(line[8]), line[1], line[4]))

In [64]:
#commented line give the quality check result with checking parameters

#query_case_3 = "SELECT * FROM songs "
query_case_3 = "SELECT firstname, lastname FROM songs "
query_case_3 +="WHERE song = 'All Hands Against His Own'"

data =[]

try:
    rows = session.execute(query_case_3)
except Exception as e:
    print(e)
    
for row in rows:
    #print (row.song, row.firstname, row.lastname,)
    
    data.append(
    
        {
            'Firstname': row.lastname,
            'Lastname': row.firstname
        }
    
    )

pd.DataFrame(data)


,Firstname,Lastname
0,Lynch,Jacqueline
1,Levine,Tegan
2,Johnson,Sara


### Drop the tables before closing out the sessions

In [65]:
query = "drop table sessions"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
    
query = "drop table users"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table songs"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [66]:
session.shutdown()
cluster.shutdown()